In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import pytesseract
from datetime import datetime
import pandas as pd
from openpyxl import load_workbook
import os

def save_results_to_excel(results_list, excel_filename):
    """Sonuçları belirtilen Excel dosyasına kaydeder veya mevcut dosyaya ekler."""
    if os.path.exists(excel_filename):
        # Dosya mevcutsa, yükle ve sonuçları altına ekle
        workbook = load_workbook(excel_filename)
        sheet = workbook.active
        # Mevcut satır sayısını al
        start_row = sheet.max_row + 1
        for result in results_list:
            sheet.append([result["Tarih"], result["Saat"], result["Plaka"], result["Çöp Atılan An Fotoğrafı"], result["Plaka Fotoğrafı"]])
        workbook.save(excel_filename)
    else:
        # Dosya yoksa, yeni bir DataFrame oluştur ve kaydet
        df = pd.DataFrame(results_list)
        df.to_excel(excel_filename, index=False)
        print(f"Sonuçlar yeni bir dosyaya kaydedildi: {excel_filename}")

# Tesseract yolu
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

object_model = YOLO('../Eğitim-dosyaları-ve-kodları/runs/detect/yolov8_envirowatch20/weights/best.pt')

plate_model = YOLO('../plaka/runs/detect/yolov11_envirowatch2/weights/best.pt')  # Plaka tanıma modeli

# Video dosyasını aç
video_path = "videolar/5.mp4"

cap = cv2.VideoCapture(video_path)

frame_count = 0  # Çerçeve sayacı

# Mesafe eşiği (piksel cinsinden)
distance_threshold = 300
#175

# Excel için sonuçların saklanacağı liste
results_list = []

def calculate_distance(box1, box2):
    """İki kutunun merkez noktaları arasındaki mesafeyi hesaplar."""
    x1_center = (box1[0] + box1[2]) // 2
    y1_center = (box1[1] + box1[3]) // 2
    x2_center = (box2[0] + box2[2]) // 2
    y2_center = (box2[1] + box2[3]) // 2
    return np.sqrt((x2_center - x1_center) ** 2 + (y2_center - y1_center) ** 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video bitti veya çerçeve okunamadı.")
        break

    frame_count += 1

    # Araç ve çöp tespiti
    results = object_model(frame)
    if results[0].boxes is None:
        continue

    vehicle_boxes = []
    trash_boxes = []

    for box in results[0].boxes:
        cls = int(box.cls[0])  # Sınıfı doğru bir şekilde alın
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        x_min, y_min, x_max, y_max = xyxy
        label = results[0].names[cls]

        if label == "car":
            vehicle_boxes.append((x_min, y_min, x_max, y_max))
        elif label == "trash":
            trash_boxes.append((x_min, y_min, x_max, y_max))

    # Araç ve çöp aynı anda mevcut mu?
    if vehicle_boxes and trash_boxes:
        for trash in trash_boxes:
            for vehicle in vehicle_boxes:
                distance = calculate_distance(trash, vehicle)

                if distance < distance_threshold:
                    print("Çöp atan araç tespit edildi!")
                    print(f"Mesafe: {distance:.2f} piksel. Çerçeve {frame_count} işleniyor.")

                    # Çöp atan aracın bulunduğu anın tarih ve saat bilgisi
                    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

                    # Kareyi kaydet
                    detection_filename = f"araclar/detected_{timestamp}_frame.jpg"
                    cv2.imwrite(detection_filename, frame)

                    # Aracın fotoğrafını kırp
                    vx_min, vy_min, vx_max, vy_max = vehicle
                    vehicle_image = frame[vy_min:vy_max, vx_min:vx_max]

                    # Plaka tespiti
                    plate_results = plate_model.predict(vehicle_image, conf=0.25, iou=0.45)
                    if plate_results[0].boxes is not None:
                        for plate_box in plate_results[0].boxes:
                            px_min, py_min, px_max, py_max = plate_box.xyxy[0].cpu().numpy().astype(int)

                            # Plakayı biraz genişlet
                            pad = 10
                            px_min = max(px_min - pad, 0)
                            py_min = max(py_min - pad, 0)
                            px_max = min(px_max + pad, vehicle_image.shape[1])
                            py_max = min(py_max + pad, vehicle_image.shape[0])

                            cropped_plate = vehicle_image[py_min:py_max, px_min:px_max]

                            # OCR ile plaka okuma
                            gray_plate = cv2.cvtColor(cropped_plate, cv2.COLOR_BGR2GRAY)
                            gray_plate = cv2.resize(gray_plate, (0, 0), fx=2, fy=2)
                            gray_plate = cv2.GaussianBlur(gray_plate, (5, 5), 0)
                            ocr_text = pytesseract.image_to_string(gray_plate, lang='eng+tur', config='--psm 7').strip()

                            # Kırpılan plaka bölgesini kaydet
                            plate_filename = f"plakalar/plate_{timestamp}_{ocr_text.replace(' ', '_')}.jpg"
                            cv2.imwrite(plate_filename, cropped_plate)

                            # Sonuçları kaydet
                            results_list.append({
                                "Tarih": timestamp.split('_')[0],
                                "Saat": timestamp.split('_')[1],
                                "Plaka": ocr_text,
                                "Çöp Atılan An Fotoğrafı": detection_filename,
                                "Plaka Fotoğrafı": plate_filename
                            })

                            print(f"Tespit Edilen Plaka: {ocr_text}")
                            print(f"Kare Kaydedildi: {detection_filename}")
                            print(f"Plaka Fotoğrafı Kaydedildi: {plate_filename}")

                            # İşlem tamamlandı
                            cap.release()

                            # Excel dosyasına yaz veya mevcut dosyaya ekle
                            excel_filename = "ceza-verileri/detected_vehicles.xlsx"
                            save_results_to_excel(results_list, excel_filename)
                            print(f"Sonuçlar {excel_filename} dosyasına kaydedildi.")
                            exit()


cap.release()
print("Video işleme tamamlandı.")



0: 640x384 2 cars, 22.0ms
Speed: 1.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 40.0ms
Speed: 2.0ms preprocess, 40.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 41.0ms
Speed: 2.0ms preprocess, 41.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 cars, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 